<a href="https://colab.research.google.com/github/surbhiagrawal22/rep0/blob/master/PDF_merger_sending_PDF_through_email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Merging Two PDF documents ( encrypt them using password) and sending via email through Python


Importing all neccassary libraries

In [26]:
import smtplib,ssl
from email import encoders
import email.mime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from getpass import getpass
import PyPDF2
from PyPDF2 import PdfFileMerger,PdfFileReader,PdfFileWriter  
from pathlib import Path

Reading the Two PDF file and merging the files into one File / encrpyt the file

In [ ]:

file_path1=Path(r'/content/BDBA_M3_DataEngineering_5_Streaming.pdf')
file_path2=Path(r'/content/Design Principles project.pdf')

file_read1=PdfFileReader(str(file_path1))
file_read2=PdfFileReader(str(file_path2))


pdf_writer=PdfFileWriter()
pdf_writer.appendPagesFromReader(file_read1) 
pdf_writer.appendPagesFromReader(file_read2)  

pdf_encrypt_pasword=getpass("Enter password for PDF encyption")
with open("UniversityProject.pdf",mode="wb") as outputfile:
  pdf_writer.encrypt(user_pwd=pdf_encrypt_pasword)
  pdf_writer.write(outputfile)
  
  print("PDF file created and encypted succesfully")

Enter password for PDF encyption··········


Creating SMTP SSL connection , Creating the Email body,attaching PDf fiel and image to Email,Sending it to receiver email address

In [ ]:
sender_email=input("Enter sender email address")
email_password=getpass("Enter Password")
receiver_email=input("Enter receiver email address")
server="smtp.gmail.com"
port=465


context=ssl.create_default_context()

message=MIMEMultipart()
message["From"]=sender_email
message["To"]=receiver_email
message["Subject"]="Please find attached combined PDF file"

htmltext="""
<html>
<p>
Hello Dear Surbhi,<br>
<br>
I have generated both files and created a combined PDF.<br>


Please find attached email<br><br>

For more information, you can check about Design principle at below link <a title="Design Principle Literature" href="https://www.interaction-design.org/literature/topics/design-principles">Design Principle</a><br><br>

Thanks Regards


</p>
</html>

"""

htmltextpart=MIMEText(htmltext,"html")
message.attach(htmltextpart)

university_project_pdf_path=Path(r'/content/UniversityProject.pdf')

with open(university_project_pdf_path,"rb") as readfile:
 part=MIMEBase("application","octet-stream")
 # read attachment file content into the MIMEBase object
 part.set_payload(readfile.read())

encoders.encode_base64(part)

# add required header data:
part.add_header(
    "Content-Disposition",
    f"attachment; filename= {university_project_pdf_path}",
)

image_path=Path(r'/content/Guide to Design Principle.png')
message.attach(image_path,"rb") as readfile:
    mime = MIMEBase('image', 'png', filename="Guide to Design Principle.png")
    # add required header data:
    mime.add_header('Content-Disposition', 'attachment', filename='Guide to Design Principle.png')
    mime.add_header('X-Attachment-Id', '0')
    mime.add_header('Content-ID', '<0>')
    # read attachment file content into the MIMEBase object
    mime.set_payload(f.read())

encoders.encode_base64(mime)

# attaching both pdf file and image object to message 
message.attach(mime)
message.attach(part)


with smtplib.SMTP_SSL(server,port=port,context=context ) as connection:
  connection.login(sender_email,email_password)
  connection.set_debuglevel(1)
  connection.sendmail(sender_email,receiver_email,message.as_string())
  print("Message send succesfully")